In [ ]:
# Import Libraries

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
# BOARD INITIALIZATION: Ruy Lopez Opening

board = np.zeros((8,8), dtype=int)

# White pieces positioning
board[0] = [1,1,1,1,1,0,0,1]
board[1] = [1,1,1,1,0,1,1,1]
board[3,4] = 1 # White Pawn: e4
board[2,5] = 1 # White Knight: f3
board[4,1] = 1 # White Bishop: b5

# Black pieces positioning
board[7] = [2,0,2,2,2,2,2,2]
board[6] = [2,2,2,2,0,2,2,2]
board[4,4] = 2 # Black Pawn: e5
board[5,2] = 2 # Black Knight: c6

In [ ]:
# CENTRAL CONTROL MATRIX: Strategic Weights

center_weight = np.array([
    [0.5, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.5],
    [0.8, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 0.8],
    [0.8, 1.2, 1.5, 1.5, 1.5, 1.5, 1.2, 0.8],
    [0.8, 1.2, 1.5, 2.0, 2.0, 1.5, 1.2, 0.8],
    [0.8, 1.2, 1.5, 2.0, 2.0, 1.5, 1.2, 0.8],
    [0.8, 1.2, 1.5, 1.5, 1.5, 1.5, 1.2, 0.8],
    [0.8, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 0.8],
    [0.5, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.5]
])

In [ ]:
# MOBILITY ENGINE: Knight & Bishop Scoring

def get_knight_score(r, c):
    """Calculates Knight mobility score (Non-linear movement, obstacle-invariant)"""
    moves = [(2,1),(1,2),(-1,2),(-2,1),(-2,-1),(-1,-2),(1,-2),(2,-1)]
    total = 0
    for dr, dc in moves:
        rr, cc = r+dr, c+dc
        if 0<=rr<8 and 0<=cc<8:
            total += center_weight[rr,cc]
    return total

def get_bishop_score(r, c, board):
    """Calculates Bishop mobility score (Linear movement, blocked by pieces)"""
    directions = [(1,1),(1,-1),(-1,1),(-1,-1)]
    total = 0
    for dr, dc in directions:
        rr, cc = r+dr, c+dc
        while 0<=rr<8 and 0<=cc<8:
            total += center_weight[rr,cc]
            if board[rr,cc] != 0: # Line of sight obstructed
                break
            rr += dr
            cc += dc
    return total

In [ ]:
# NORMALIZATION: Maximum Potential on Ideal Board

IDEAL_BOARD = np.zeros((8,8)) # Ideal, unobstructed board for theoretical bishop mobility
center_sq = [(3,3),(3,4),(4,3),(4,4)]
max_k = np.mean([get_knight_score(r,c) for r,c in center_sq])
max_b = np.mean([get_bishop_score(r,c,IDEAL_BOARD) for r,c in center_sq])

In [ ]:
# COMPARATIVE ANALYSIS: Relative Piece Efficiency

top1_map = np.empty((8,8), dtype=object)
threshold = 0.005 # decision margin

for r in range(8):
    for c in range(8):
        # Efficiency Ratio = Current Performance / Theoretical Potential
        k_perf = get_knight_score(r,c) / max_k
        b_perf = get_bishop_score(r,c,board) / max_b
        diff = b_perf - k_perf
        if diff > threshold: top1_map[r,c] = "Bishop"
        elif diff < -threshold: top1_map[r,c] = "Knight"
        else: top1_map[r,c] = "Equal"

In [ ]:
# VISUALIZATION: Comparative Efficiency

symbol = {"Bishop":"♝","Knight":"♞","Equal":"—"}
colors = {"Bishop": (1,0.84,0), "Knight": (0,0.7,1), "Equal": (0.5,0.5,0.5)}

fig, ax = plt.subplots(figsize=(10,10), facecolor='#111111')
ax.set_facecolor('#111111')

for r in range(8):
    for c in range(8):
        res = top1_map[r,c]
        if board[r,c] != 0: # Mark occupied squares
            rect_color = (0.3,0.3,0.3,0.2)
            text_sym = "★"
            text_color = "lightgray"
        else: # Efficiency indicators for empty squares
            rect_color = list(colors[res]) + [0.2]
            text_sym = symbol[res]
            text_color = "white"
        
        rect = patches.Rectangle((c-0.5,r-0.5),1,1, facecolor=rect_color, edgecolor='none')
        ax.add_patch(rect)
        ax.text(c,r,text_sym,ha='center',va='center',color=text_color,fontsize=30,weight='bold')

ax.set_xticks(np.arange(8))
ax.set_yticks(np.arange(8))
ax.set_xticklabels(['A','B','C','D','E','F','G','H'], color='white', fontsize=14)
ax.set_yticklabels(['1','2','3','4','5','6','7','8'], color='white', fontsize=14)
ax.set_xlim(-0.5,7.5)
ax.set_ylim(-0.5,7.5)
ax.set_xticks(np.arange(-0.5,8,1), minor=True)
ax.set_yticks(np.arange(-0.5,8,1), minor=True)
ax.grid(which='minor', color='white', linestyle='-', linewidth=1, alpha=0.1)
main_title = "Relative Positional Efficiency: Bishop vs Knight"
note_title = "(Ruy Lopez Opening Structure)"
ax.set_title(main_title, color='white', fontsize=18, pad=35, fontweight='bold')
ax.text(0.5, 1.02, note_title, transform=ax.transAxes, ha='center', va='bottom',
        color='#cccccc', fontsize=13, style='italic')

plt.tight_layout()
plt.show()